# SDG 11.2.1 (mh edit)

## 11.2.1
Proportion of population that has convenient access to public transport, by sex, age and persons with disabilities 

**11.2.1 is under 11.2**

## 11.2
By 2030, provide access to safe, affordable, accessible and sustainable transport systems for all, improving road safety, notably by expanding public transport, with special attention to the needs of those in vulnerable situations, women, children, persons with disabilities and older persons 


Method
<ol>
1) get public transport location data - NAPTAN a) Clean it if necessary
2) get population location data -  LSOA from ONS
3) use Fiona to read location data
4) limit to one or two locations, e.g. London and a more rural area
5) draw Euclidean Buffers around LSOA polygon centre points
6) find number of public transport stops in the polygon with “points in polygons” approach
</ol>

Requirements: rtree you need to install rtree 'pip install rtree'

In [15]:
%load_ext pycodestyle_magic
%pycodestyle_on

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [16]:
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import requests
import json
import io
import pyproj

In [29]:
def geo_df_from_csv(path_to_csv, geom_x, geom_y, delim='\t', crs ="EPSG:4326"):
    """Function to create a Geo-dataframe from a csv file.
        The process goes via Pandas
    
        Arguments:
            path_to_csv (string): path to the txt/csv containing geo data
                to be read
            delimiter (string): the seperator in the csv file e.g. "," or "\t" 
            geom_x (string):name of the column that contains the longitude data
            geom_y (string):name of the column that contains the latitude data
            
        Reurns:
            Geopandas Dataframe
            """
    pd_df = pd.read_csv(path_to_csv, delim)
    geometry = [Point(xy) for xy in zip(pd_df[geom_x], pd_df[geom_y])]
    geo_df = gpd.GeoDataFrame(pd_df, geometry=geometry)
    geo_df.crs = crs
    return geo_df


stops_path = (os.path.join
              (os.getcwd(),
               'data',
               'Stops.txt'))

stops_geo_df = (geo_df_from_csv(path_to_csv=stops_path,
                            delim='\t',
                            geom_x='stop_lon',
                            geom_y='stop_lat'))
stops_geo_df.sample(15)

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,vehicle_type,geometry
179633,300000615A,nthdgpmj,"Farthinghoe, Almshouses (o/s)",52.05284,-1.21804,NaN,3.0,POINT (-1.21804 52.05284)
230746,390010700,sufjdmap,"Great Cornard, Beech Road (adj)",52.03114,0.75123,NaN,3.0,POINT (0.75123 52.03114)
33661,079073039B,mbrajgj,"Linthorpe, Masonic Hall (2)",54.55480,-1.24604,NaN,3.0,POINT (-1.24604 54.55480)
179424,300000578S,nthdawaj,"Welland Vale, Welland Vale Road Shops (opp)",52.50218,-0.70865,NaN,3.0,POINT (-0.70865 52.50218)
299727,490009322S2,76009,"Finchley, Long Lane Finchley Central (Stop L)",51.60439,-0.18816,NaN,3.0,POINT (-0.18816 51.60439)
298236,490008069W,54836,"Finchley, Henlys Corner (Stop TZ)",51.58957,-0.19822,NaN,3.0,POINT (-0.19822 51.58957)
287230,4600WOZ03957,wilapgtj,"Stanley, Tytherton Turn (NE-bound)",51.45524,-2.06335,NaN,3.0,POINT (-2.06335 51.45524)
263202,4400EG0324,wsxapjmd,"East Grinstead, Lingfield Road (adj)",51.13156,-0.01957,NaN,3.0,POINT (-0.01957 51.13156)
22198,0590PBE661,PETPGDT,"Etton, Bus Shelter (Adj)",52.64296,-0.31545,NaN,3.0,POINT (-0.31545 52.64296)
162693,2700LBLE4930,lindpgaj,"Asgarby, Cottages (adj)",52.99760,-0.32651,NaN,3.0,POINT (-0.32651 52.99760)


1:65: E251 unexpected spaces around keyword / parameter equals
4:1: W293 blank line contains whitespace
8:79: W291 trailing whitespace
11:1: W293 blank line contains whitespace
28:29: E128 continuation line under-indented for visual indent
29:29: E128 continuation line under-indented for visual indent
30:29: E128 continuation line under-indented for visual indent
1:65: E251 unexpected spaces around keyword / parameter equals
4:1: W293 blank line contains whitespace
8:79: W291 trailing whitespace
11:1: W293 blank line contains whitespace
28:29: E128 continuation line under-indented for visual indent
29:29: E128 continuation line under-indented for visual indent
30:29: E128 continuation line under-indented for visual indent


In [35]:
# Load Greater London polygon and check CRS is 4326
greater_london = (gpd.read_file(os.path.join
                                (os.getcwd(),
                                 'data',
                                 'greater_london.geojson')))
greater_london.crs

,id,EER13CD,EER13CDO,EER13NM,geometry
0,E15000007,E15000007,07,London,"MULTIPOLYGON (((-0.32111 51.44603, -0.32520 51..."


In [74]:
def find_points_in_poly(geo_df, polygon_obj):
    """Find points in polygon using geopandas' spatial join.
        Then drops all rows where the point is not in the polygon
        (based on column index_right not being NaN). Finally it
        drop all column names from that were created in the join,
        leaving only the columns of the original geo_df
        
        Arguments:
            geo_df (string): name of a geo pandas dataframe
            polygon_obj (string): a geopandas dataframe with a polygon column
            
        Returns:
            A geodata frame with the points inside the supplied polygon"""
    wanted_cols = geo_df.columns.to_list()
    joined_df = (gpd.sjoin
                 (geo_df,
                  polygon_obj,
                  how='left',
                  op='within'))
    filtered_df = (joined_df
                   [joined_df
                    ['index_right'].notna()])
    filtered_df = filtered_df[wanted_cols]
    return filtered_df


# Creating a Geo Dataframe of only stops in London
london_stops_geo_df = (find_points_in_poly
                       (geo_df=stops_geo_df,
                        polygon_obj=greater_london))

/home/james/anaconda3/envs/SDG_11.2.1/lib/python3.8/site-packages/geopandas/tools/sjoin.py:58: UserWarning: CRS of frames being joined does not match!(EPSG:4326 != {'init': 'epsg:4326'})
  warn(
5:66: W291 trailing whitespace
7:1: W293 blank line contains whitespace
11:1: W293 blank line contains whitespace
14:16: E221 multiple spaces before operator
27:1: E305 expected 2 blank lines after class or function definition, found 1
5:66: W291 trailing whitespace
7:1: W293 blank line contains whitespace
11:1: W293 blank line contains whitespace
14:16: E221 multiple spaces before operator
27:1: E305 expected 2 blank lines after class or function definition, found 1


In [78]:
# Building the map of Local Authority Districts

Birmingham_map_df = (gpd.read_file
                  (os.path.join
                   (os.getcwd(),
                    'data',
                    'Birmingham_merged_census_BoundaryData',
                    'england_oac_2011.shp')))

# # locAuth_map_df.to_crs(london_stops_geo_df.crs)
# Birmingham_map_df.to_crs("EPSG:4326")
# gpd.__version__

Birmingham_map_df.head()

,oac_sub_gr,oac_group_,oac_super_,oac_sub__1,oac_group,oac_supe_1,name,label,code,geometry
0,Student Digs,Students Around Campus,Cosmopolitans,2a2,2a,2,None,E08000025E02001890E01008987E00045661,E00045661,"POLYGON ((404630.067 286620.263, 404643.909 28..."
1,Student Digs,Students Around Campus,Cosmopolitans,2a2,2a,2,None,E08000025E02001890E01008987E00045660,E00045660,"POLYGON ((404621.248 286626.915, 404619.220 28..."
2,Student Digs,Students Around Campus,Cosmopolitans,2a2,2a,2,None,E08000025E02001922E01009290E00047058,E00047058,"POLYGON ((405051.995 282349.537, 405052.241 28..."
3,Student Digs,Students Around Campus,Cosmopolitans,2a2,2a,2,None,E08000025E02001922E01009289E00047057,E00047057,"POLYGON ((404631.158 283001.345, 404633.917 28..."
4,Student Digs,Students Around Campus,Cosmopolitans,2a2,2a,2,None,E08000025E02001922E01009286E00047060,E00047060,"POLYGON ((404914.500 282660.703, 404916.036 28..."


4:19: E128 continuation line under-indented for visual indent
4:19: E128 continuation line under-indented for visual indent


In [13]:
# def get_geo_dataset(filename, url, localpath):
#     path_to_local_file = os.path.join(localpath, filename)
#     if os.path.exists(path_to_local_file):
#         file = path_to_local_file   
#     else:
#         file = requests.get(url)
#     return file

# url = "https://raw.githubusercontent.com/ONSvisual/topojson_boundaries/master/LSOA.json"
    

    
# data = requests.get(url)
# file_path = os.path.join('data', 'LSOA.json')
# with open(file_path, 'ab') as geo_file:
#     geo_file.write(data.content)
# #     gdf = gpd.GeoDataFrame(geo_file.json())

# # # df = pd.read_json(io.BytesIO(file.content))

# # gdf = gpd.GeoDataFrame(open(file_path).json())

# # type(file_path)
# # file_path
# df = gpd.read_file(open(file_path, 'rb'), driver='GeoJSON')


4:36: W291 trailing whitespace
9:80: E501 line too long (90 > 79 characters)
10:1: W293 blank line contains whitespace
12:1: W293 blank line contains whitespace
13:1: E303 too many blank lines (3)
26:1: W391 blank line at end of file


In [14]:
## Testing that I can filter based on placename, e.g. Islington

locAuth_map_df[locAuth_map_df['lad17nm']=='Islington']
## works!

NameError: name 'locAuth_map_df' is not defined

1:1: E266 too many leading '#' for block comment
3:41: E225 missing whitespace around operator
4:1: E266 too many leading '#' for block comment


In [ ]:
# likewise, I can filter on the lad17cd code. I will use the code to filter for places in London
## I was told 'E09' is for London

locAuth_map_df[locAuth_map_df.lad17cd.str.contains('E09')]

In [ ]:
## Making a filtered geo_df of just the places with 'E09' in their string
## rathter than using str.contains, I'll use == instead
lond_districts = locAuth_map_df[locAuth_map_df.lad17cd.str[:3]=='E09']

## PLotting the london districts just to see how it looks
_ = lond_districts.plot()

In [ ]:
## checking the new london districts df

lond_districts.head()

In [ ]:
# Make a polygon to border the whole of London
## make a temp dataframe
_ = lond_districts
## make an extra column 'city_name' on which to dissolve on, making them all the same value, 'London'
_.loc[:,'city_name'] = 'London'
# ## make the polygon with dissolve, using city_name
whole_london_poly = _.dissolve(by='city_name')
## Seems like a bit of a bizzare method, but it works, I think

In [ ]:
## Making a centroid 
centrepoint = whole_london_poly.centroid

ward = 'Brent'

ward_polygon = lond_districts[lond_districts.lad17nm == ward]
## plot the whole_london_poly polygon 
## with the selected stops laid over it 

fig, ax = plt.subplots()
_ =  lond_districts.plot(ax=ax)
_ = ward_polygon.plot(ax=ax, facecolor='gold')
_ = london_stops_geo_df.plot(ax=ax, color='red', markersize=2, alpha=0.1)
_ = centrepoint.plot(ax=ax, color='pink', markersize=45) ## added the centroid into the plot
plt.tight_layout()

## great, this works!

In [ ]:
# Filter for ward

ward_stops_geo_df = gpd.sjoin(london_stops_geo_df, Brent, how='left', op='within')
ward_stops_geo_df.head()

# Drop all rows where id (from polygon) is NaN, that is, where the point is not in the polygon

ward_stops_geo_df = ward_stops_geo_df[ward_stops_geo_df['index_right'].notna()]
ward_stops_geo_df

# Drop all row names from join (so we can reuse)

ward_stops_geo_df = ward_stops_geo_df[stops_geo_df.columns.to_list()]
ward_stops_geo_df

In [ ]:
## Making a centroid 
centrepoint = ward_polygon.centroid

fig, ax = plt.subplots()
_ = ward_polygon.plot(ax=ax, facecolor='gold')
_ = ward_stops_geo_df.plot(ax=ax, color='red', markersize=2, alpha=0.1)
_ = centrepoint.plot(ax=ax, color='pink', markersize=45) ## added the centroid into the plot
plt.tight_layout()

## great, this works!